# PREPROCESSING POINT CLOUD DATA TO LINKED DATA
In this notebook, we evaluate the point cloud assets in a session repository.
For every e57 point cloud with its accompanying xml file, a POINTCLOUDNODE metadata class is created 
that governs all the metadata of the point cloud (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) and O3D.GEOMETRY.POINTCLOUD (.pcd) files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [1]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

import os.path, time
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## 1. INITIALIZE SESSION

In [4]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"test" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath,"testfiles") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
graphPath = os.path.join(sessionPath,"pcdGraph.ttl")

sensor='Leica Scanstation P30'
accuracy=0.002

In [5]:
# # run e57xmldump.exe on e57 files:THIS CURRENTLY DOESN4T WORK
# command = ""
# command = "D:\\Scan-to-BIM repository\\Scan-to-BIM-Grasshopper\\Scan2BIM\\4.Python\\e57xmldump "
# command = command + pcdfolderpath+filepath
# command = command + " > " + pcdfolderpath+filepath.replace('.e57', '.xml') 
# command = ".\e57xmldump" +' "'+filepath+'" ' +">"+ ' "'+filepath.replace('.e57', '.xml') +'" '

# # os.system(command)
# #os.system(".\e57xmldump D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week 22 lidar.e57 > D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week 22 lidar.xml")
# # there is an issue with this command
# # import subprocess
# # subprocess.run(["e57xmldump", "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week22_photogrammetry_densecloud - Cloud.e57","D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week22_photogrammetry_densecloud - Cloud.xml"],shell=True)
# # THIS COMMAND WORKS IN CMD BUT NOT WITH OS.SYSTEM
# # command = '.\e57xmldump "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week22 photogrammetry - Cloud.e57" > "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week22 photogrammetry - Cloud.xml"'
# # os.system(command)
# .\e57xmldump "D:\\Data\\2021-07 Peter Benoitlaan\\PCD\\Peter Benoitlaan 16 1.e57" > "D:\\Data\\2021-07 Peter Benoitlaan\\PCD\\Peter Benoitlaan 16 1.xml"

## 2. READ PRESENT E57 XML/.E57 FILES CONTAINING POINT CLOUDS
A PointCloudNode is created per e57 point cloud scan in the session
This codes presumes that .e57xmldump.exe has generated an .xml metadata file for every e57 in the session
Also, the .xml should not contain <?xml version="1.0" encoding="UTF-8"?>
E.g.: .\e57xmldump "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week22 photogrammetry - Cloud.e57" > "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week22 photogrammetry - Cloud.xml"
.\e57xmldump "K:\Projects\2024-10 Project FWO Postdoc MB\6.Code\GEOMAPITICS\tests\Samples\\pointcloud.e57" > "K:\Projects\2024-10 Project FWO Postdoc MB\6.Code\GEOMAPITICS\tests\Samples\\pointcloud.xml"

In [6]:
nodelist=[]

pcdPath=os.path.join(sessionPath,'PCD')
allSessionFilePaths=ut.get_list_of_files(pcdPath) 
for path in allSessionFilePaths:
    if path.endswith(".e57"): 
        print(path)        
        list=tl.e57header_to_nodes(path,getResource=True) 
        nodelist.extend(list)       
    if path.endswith('.pcd'):   
        print(path) 
        nodelist.append(PointCloudNode(path=path,getResource=True))

d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\academiestraat week 22 19a.pcd
d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\academiestraat week 22 20a.pcd
d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\navvis.e57
d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\week 22 - Lidar.e57
d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\week22 photogrammetry - Cloud.e57
d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\week22 photogrammetry - Cloud.pcd
d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\week22_photogrammetry_densecloud - Cloud.e57


In [7]:
# Retrieve all (.e57, e57xml and pcd) files in the session
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
e57Counter=0
pcdCounter=0
nodelist=[]

for path in allSessionFilePaths:
    if path.endswith(".e57"): 
        list=[]
        xmlPath=path.replace('.e57','.xml')
        if xmlPath in allSessionFilePaths:
            print('e57 xml file found: '+path)   
            list=tl.e57xml_to_nodes(xmlPath,getResource=True,sessionPath=sessionPath)
        if list is None or len(list)==0:
            print('e57 file found: '+path)   
            list=tl.e57header_to_nodes(path,getResource=True,sessionPath=sessionPath)        
        if list is not None:
            e57Counter=e57Counter+len(list) 
            nodelist.extend(list)
    # if path.endswith('.pcd'):         
    #     pcdPath=path.replace('.pcd','.e57')
    #     if pcdPath not in allSessionFilePaths:
    #         print('pcd file found: '+path)
    #         pcdCounter+=1
    #         nodelist.append(tl.pcd_to_node(path))           

# for node in nodelist:
#     node.sessionPath=sessionPath

print(str(len(nodelist))+ ' PointCloudNodes are created:')
print(str(e57Counter) +'/'+str(len(nodelist))+' from e57 files')
print(str(pcdCounter) +'/'+str(len(nodelist))+' from pcd files')
print('these nodes do not yet contain data but only the metadata')

e57 file found: d:\Scan-to-BIM repository\geomapi\test\testfiles\e57.e57
e57 file found: d:\Scan-to-BIM repository\geomapi\test\testfiles\PCD\navvis.e57


In [5]:
for node in nodelist:
    if 'l72'in node.name:
        node.coordinateSystem='Lambert72'
    elif 'l2008' in node.name:
        node.coordinateSystem='Lambert2008'
    elif 'wgs84' in node.name:
        node.coordinateSystem='geospatial-wgs84'
    else:
        node.coordinateSystem='local'
print('Node coordinate systems have been set based on file names (l72, l2008, wgs84, local)!')

Node coordinate systems have been set based on file names (l72, l2008, wgs84, local)!


## 3. WRITE PCD FILES 
A .pcd file is created per lidar scan in the session
already processed .pcd files are not overwritten

In [6]:
#Read e57 files, create o3d point clouds and write them to \\PCD\\'pcdnode.name'.pcd
newPcdCounter=0
oldPcdCounter=0
for node in nodelist:
    if node.path is not None and os.path.exists(node.path):
        oldPcdCounter+=1
        pass
    else:
        node.set_pcd_path_from_e57()
        node.get_geometry()
        o3d.io.write_point_cloud(node.path, node.pcd)
        newPcdCounter+=1
       
print (str(oldPcdCounter)+' of '+str(len(nodelist))+ ' PointCloudNodes already have data!')
print (str(newPcdCounter)+' of '+str(len(nodelist))+ ' PointCloudNodes newly created!')

0 of 4 PointCloudNodes already have data!
4 of 4 PointCloudNodes newly created!


## 4. CREATE RDF GRAPHS FOR EACH POINTCLOUDNODES AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all point clouds in the session
(data itself is not stored in the graph, only metadata)

In [7]:
graph=nodelist[2].to_graph(graphPath)
print(graph.serialize())

@prefix e57: <http://libe57.org#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///_65FBBFC3-1192-47C2-BCC1-B2BF66840C4A_-Cloud-1> a v4d:PointCloudNode ;
    e57:cartesianBounds """[-14.56856251  18.11331177 -16.01319885  15.32858181  -1.11594343
  15.32411003]""" ;
    e57:cartesianTransform """[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]""" ;
    e57:e57Index 0 ;
    e57:pointCount 20168806 ;
    v4d:name "{65FBBFC3-1192-47C2-BCC1-B2BF66840C4A}-Cloud-1" ;
    v4d:orientedBounds """[[-13.47140023 -17.40796858  -0.17794121]
 [ 19.16010023 -15.45181523  -1.25154832]
 [-15.44981412  13.51956681  -3.95922326]
 [-13.02567042 -15.24017258  17.31949716]
 [ 17.62741617  17.64351618  12.46460799]
 [-15.0040843   15.68736282  13.5382151 ]
 [ 19.60583005 -13.28401922  16.24589004]
 [ 17.18168635  15

In [274]:
# print(graph.serialize())
g=Graph()
g=ut.bind_ontologies(g)
g+=nodelist[0].to_graph(graphPath)
g+=nodelist[1].to_graph(graphPath)
g+=nodelist[2].to_graph(graphPath)


# for node in nodelist:
#     node.to_graph(graphPath)
#     g+= node.graph
print(len(g))
print(g.serialize())

21


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
graph=tl.nodes_to_graph(nodelist=nodelist,graphPath=graphPath,save=True)
print(len(graph))
print(graph.serialize())
print (str(len(nodelist))+' Nodes succesfully serialized in: '+graphPath)


65
@prefix e57: <http://libe57.org#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///_65FBBFC3-1192-47C2-BCC1-B2BF66840C4A_-Cloud-1> a v4d:PointCloudNode ;
    e57:cartesianBounds """[-14.56856251  18.11331177 -16.01319885  15.32858181  -1.11594343
  15.32411003]""" ;
    e57:cartesianTransform """[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]""" ;
    e57:e57Index 0 ;
    e57:pointCount 20168806 ;
    v4d:name "{65FBBFC3-1192-47C2-BCC1-B2BF66840C4A}-Cloud-1" ;
    v4d:orientedBounds """[[-13.47140023 -17.40796858  -0.17794121]
 [ 19.16010023 -15.45181523  -1.25154832]
 [-15.44981412  13.51956681  -3.95922326]
 [-13.02567042 -15.24017258  17.31949716]
 [ 17.62741617  17.64351618  12.46460799]
 [-15.0040843   15.68736282  13.5382151 ]
 [ 19.60583005 -13.28401922  16.24589004]
 [ 17.18168635 